## Final Project Submission

Please fill out:
* Student name: George Gachugu Kariuki
* Student pace: full time
* Scheduled project review date/time: 22 March 2024
* Instructor name: Asha Deen
* Blog post URL:

# Title - Microsoft Movies

## Business Overview
Microsoft Studios have are excited to delve into the world of creative arts videos and movie production. To aid the head of Microsoft Movies I ahve created this project that attempts to undertand the movies at the box office with an aim to explore the kind of films that are doing well at the box office. Some of the questions we shall explore and investigate include:
### 1. Genres 
    - What are the best performing genres at the box office?
### 2. Directors - Best performing directors by revenue and volume
    - Who are the best performing directors at the box office?
### 3. Studios - Best performing studios by revenue and volume
    - Which studios are the best performing studios at the box office?
    
Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
%matplotlib inline

## Data Understanding
We have two sources of data: a sqlite3 database that contains information on the movies in different tables, and a csv files from imdb.

### Sqlite3 Databases
There are 5 tables in the database - 3 of which we will use in our analysis - Movie basics, Directors and the Movie Ratings tables.
In line with our objective, I joined the 3 tables based on the common feature `movie_id` to form one table - picking only the relevant features avoiding any duplications. These features are the movie_id, original_title	primary_name, runtime_minutes, genres, averagerating and numvotes.
Using the Pandas library, I transformed the table into a Pandas DataFrame for easier manipulation

In [7]:
# Creating a connection object using sqlite3 library
conn = sqlite3.connect("./zippedData/im.db/im.db")

In [12]:
pd.read_sql("SELECT * FROM sqlite_master;", conn).head()

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."


In [45]:
query = """
    SELECT original_title as title, primary_name AS director, 
        runtime_minutes, genres
    FROM directors
    JOIN persons
        USING (person_id)
    JOIN movie_basics
        USING (movie_id)
    JOIN movie_ratings
        USING (movie_id)
"""
im_df = pd.read_sql(query, conn)
im_df.tail()

,original_title,primary_name,runtime_minutes,genres,averagerating,numvotes
181382,Reversing Roe,Anne Sundberg,99.0,Documentary,7.4,696
181383,The Princess Switch,Mike Rohl,101.0,Romance,6.0,13993
181384,The Princess Switch,Mike Rohl,101.0,Romance,6.0,13993
181385,Doozy,Richard Squires,70.0,"Animation,Comedy",6.7,13
181386,Syndebukken: Prosessen mot Harry Lindstrøm,Fredrik Horn Akselsen,NaN,Documentary,8.4,7


In [69]:
im_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181387 entries, 0 to 181386
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   original_title   181387 non-null  object 
 1   primary_name     181387 non-null  object 
 2   runtime_minutes  163584 non-null  float64
 3   genres           180047 non-null  object 
 4   averagerating    181387 non-null  float64
 5   numvotes         181387 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 8.3+ MB


### CSV Files
The [first csv file]('./zippedData/bom.movie_gross.csv.gz') contains the movies' titles, studios, gross earnings in domestic and foreign currencies, and the year released or produced. This file had 3387 records in it.

The [second csv file]('./zippedData/tmdb.movies.csv.gz') contains the features - genre_ids, id, original_language, original_title, popularity, release_date, title, vote_average and vote_count. This file contained 26517 records in it.

Lastly, we obtain our budgets from the [budgets csv]('./zippedData/tn.movie_budgets.csv.gz') and other data from this dataset. This dataset contained 5782 records with 6 features - id, release date, movie, production_budget, domestic_gross, worldwide_gross.

BOM Data

In [46]:
bom_df = pd.read_csv("./zippedData/bom.movie_gross.csv.gz")
print(bom_df.shape)
bom_df.head()

(3387, 5)


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [66]:
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


TMDB Data

In [49]:
tmdb_df = pd.read_csv("./zippedData/tmdb.movies.csv.gz", index_col=0)
print(tmdb_df.shape)
tmdb_df.head()

(26517, 9)


,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [68]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


## Data Preparation

### Data Cleaning
Before merging, we need to clean the data from the bom dataset. The foreign_gross column is an object whereas it shoul be an a float

In [60]:
# tmdb_df.groupby('original_language')['original_title'].count().value_counts(ascending=False)
tmdb_df.original_language.value_counts(ascending=False)

en    23291
fr      507
es      455
ru      298
ja      265
      ...  
bo        1
hy        1
hz        1
xh        1
yi        1
Name: original_language, Length: 76, dtype: int64

In [35]:
# Drop unnecessary columns in tmdb_df
tmdb_df.drop(columns=["genre_ids", "id", "original_language", "original_title"], inplace=True)

In [36]:
bomtmdb = pd.merge(left=bom_df, right=tmdb_df, how="inner", on="title")

In [40]:
print(bomtmdb.shape)
bomtmdb.head()

(2703, 9)


,title,studio,domestic_gross,foreign_gross,year,popularity,release_date,vote_average,vote_count
0,Toy Story 3,BV,415000000.0,652000000,2010,24.445,2010-06-17,7.7,8340
1,Inception,WB,292600000.0,535700000,2010,27.920,2010-07-16,8.3,22186
2,Shrek Forever After,P/DW,238700000.0,513900000,2010,15.041,2010-05-16,6.1,3843
3,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010,20.340,2010-06-23,6.0,4909
4,Iron Man 2,Par.,312400000.0,311500000,2010,28.515,2010-05-07,6.8,12368
